<a href="https://colab.research.google.com/github/william-toscani/Data_Visualization_Project/blob/main/Data_Viz_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libraries

In [ ]:
#!pip install pandasql

In [2]:
# Data managment
import pandas as pd
from pandasql import sqldf
sql = lambda q: sqldf(q, globals())

# Data Visualization
import plotly.express as px

# Datasets

## 1. World Dataset (Country, Code, Region)

In [3]:
world_raw = pd.read_csv("https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv")
world_raw.rename(columns={'alpha-3': 'code'}, inplace=True)

world_clean = sql("""
SELECT name, code, region FROM world_raw
""")

display(world_clean)

,name,code,region
0,Afghanistan,AFG,Asia
1,Åland Islands,ALA,Europe
2,Albania,ALB,Europe
3,Algeria,DZA,Africa
4,American Samoa,ASM,Oceania
...,...,...,...
244,Wallis and Futuna,WLF,Oceania
245,Western Sahara,ESH,Africa
246,Yemen,YEM,Asia
247,Zambia,ZMB,Africa


## 2. GDP Dataset (Country GDP)

In [42]:
gdp_raw = pd.read_csv("https://ourworldindata.org/grapher/gdp-worldbank.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
#gdp_metadata = requests.get("https://ourworldindata.org/grapher/gdp-worldbank.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

gdp_clean = sql("""
SELECT wc.code, wc.name, ny_gdp_mktp_pp_kd as gdp FROM gdp_raw
INNER JOIN world_clean as wc ON gdp_raw.Code = wc.code
WHERE year = 2024 AND wc.region IS "Europe"
ORDER BY gdp DESC
""")

display(gdp_clean)

,code,name,gdp
0,RUS,Russian Federation,6088997179872
1,DEU,Germany,5246975952526
2,FRA,France,3731760755094
3,GBR,United Kingdom of Great Britain and Northern I...,3635610019744
4,ITA,Italy,3133050580636
5,ESP,Spain,2360949862238
6,POL,Poland,1649077853050
7,NLD,"Netherlands, Kingdom of the",1275824484216
8,ROU,Romania,774375748379
9,BEL,Belgium,749229139749


In [44]:
fig_gdp_europe = px.choropleth(gdp_clean, locations='code', color='gdp',
                    hover_name='name', projection="natural earth",
                    color_continuous_scale=px.colors.sequential.Plasma,
                    title='GDP per European Country (2024)',
                    scope = 'europe',
                    fitbounds='locations')
fig_gdp_europe.update_layout(width=1000, height=800, title_x=0.5) # Make the plot square and center the title
fig_gdp_europe.show()

## 3. Co2 emissions Dataset

In [40]:
emission_raw = pd.read_csv("https://ourworldindata.org/grapher/annual-co2-emissions-per-country.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
#emission_metadata = requests.get("https://ourworldindata.org/grapher/annual-co2-emissions-per-country.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()
#display(emission_raw.head())

emission_clean = sql("""
SELECT wc.region, wc.code, emissions_total FROM emission_raw
INNER JOIN world_clean as wc ON emission_raw.Code = wc.code
INNER JOIN gdp_clean ON emission_raw.Code = gdp_clean.code
WHERE year = 2024 AND wc.region IS "Europe"
ORDER BY emissions_total DESC
""")

display(emission_clean)

,region,code,emissions_total
0,Europe,RUS,1.780524e+09
1,Europe,DEU,5.723192e+08
2,Europe,GBR,3.129055e+08
3,Europe,ITA,3.019298e+08
4,Europe,POL,2.728618e+08
5,Europe,FRA,2.641556e+08
6,Europe,ESP,2.203413e+08
7,Europe,UKR,1.424979e+08
8,Europe,NLD,1.147847e+08
9,Europe,BEL,8.545620e+07


## 4. Share energy Database

In [55]:
share_raw = pd.read_csv("https://ourworldindata.org/grapher/share-of-primary-energy-consumption-by-source.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
#metadata = requests.get("https://ourworldindata.org/grapher/share-of-primary-energy-consumption-by-source.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

fossil_columns = ['gas', 'oil', 'coal']
renew_cols = ['wind', 'hydro', 'other_renewables', 'nuclear', 'solar', 'biofuels']

new_columns = {}
for col in share_raw.columns:
    if '__pct_direct_primary_energy' in col:
        new_name = col.replace('__pct_direct_primary_energy', '')
        new_columns[col] = new_name
share_raw = share_raw.rename(columns=new_columns)

share_clean = sql(f"""
SELECT wc.name, wc.code, {",".join(fossil_columns)}, {",".join(renew_cols)} FROM share_raw
INNER JOIN gdp_clean ON share_raw.Code = gdp_clean.code
INNER JOIN world_clean as wc ON share_raw.Code = wc.code
WHERE year = 2024 AND wc.region IS "Europe"
""")

display(share_clean)

,name,code,gas,oil,coal,wind,hydro,other_renewables,nuclear,solar,biofuels
0,Austria,AUT,22.632970,43.752766,9.277962,3.055430,14.861383,1.541471,0.000000,3.126327,1.751691
1,Belarus,BLR,58.215400,32.091440,3.866252,0.059731,0.093311,0.176509,5.445542,0.051812,0.000000
2,Belgium,BEL,23.862420,58.838610,5.511098,2.371115,0.095255,0.583786,5.618127,1.545744,1.573844
3,Bulgaria,BGR,18.176176,40.994648,21.195766,0.922531,1.927641,1.153046,10.557089,3.631812,1.441292
4,Croatia,HRV,27.951710,55.583473,3.128219,3.166354,7.996201,1.196350,0.000000,0.966420,0.011271
5,Czechia,CZE,20.539380,34.668716,30.659160,0.216452,0.815642,1.688485,9.118636,1.102470,1.191059
6,Denmark,DNK,11.302169,62.446857,3.672187,14.208114,0.013487,4.003976,0.000000,2.692655,1.660555
7,Estonia,EST,6.877415,31.016407,54.334740,2.127328,0.064310,2.802327,0.000000,2.287264,0.490210
8,Finland,FIN,6.143715,42.058080,10.145653,10.021552,6.946723,4.978643,15.829950,0.621486,3.254199
9,France,FRA,18.694588,45.359646,2.887011,2.756961,4.146557,0.562129,22.248041,1.355730,1.989337


# Data Analysis

## Vediamo i 10 paesi che hanno un'efficenza ambientale economica elevata

In [ ]:
top_10_emissions_gdp = sql("""
SELECT wc.name, wc.region, ec.emissions_per_gdp FROM emissions_gdp AS ec
INNER JOIN world_clean AS wc ON ec.Code = wc.Code
ORDER BY emissions_per_gdp
LIMIT 10
""")

display(top_10_emissions_gdp)

In [ ]:
emissions_by_continent = sql("""
SELECT region, SUM(emissions_total) AS total_emissions FROM emission_clean
GROUP BY region
ORDER BY total_emissions ASC
""")

display(emissions_by_continent)

## 0. Andiamo a vedere tra i 100 paesi con piu popolazione i 20 paesi che emettono di piu e di meno rispetto al loro gdp e vediamo come è composto il loro mix energetico

In [ ]:
top_20 = sql("""
SELECT sc.fossil, sc.renewable, wc.name, wc.region, ec.emissions_per_gdp FROM emissions_gdp AS ec
INNER JOIN world_clean AS wc ON ec.Code = wc.Code
INNER JOIN pop_clean AS pc ON ec.Code = pc.Code
LEFT JOIN share_clean AS sc ON ec.Code = sc.Code
ORDER BY emissions_per_gdp DESC
LIMIT 20
""")

flop_20 = sql("""
SELECT sc.fossil, sc.renewable, wc.name, wc.region, ec.emissions_per_gdp FROM emissions_gdp AS ec
INNER JOIN world_clean AS wc ON ec.Code = wc.Code
INNER JOIN pop_clean AS pc ON ec.Code = pc.Code
LEFT JOIN share_clean AS sc ON ec.Code = sc.Code
ORDER BY emissions_per_gdp ASC
LIMIT 20
""")
display(top_20, flop_20)

## 1. Andiamo a confrontare le 20 economie piu grandi del mondo (per gdp) del 2024 e andiamo a vedere in che posizioni si trovano per quanto riguarda le emissioni di c02 prodotte

In [ ]:
gdp_top20 = sql("""
SELECT  wc.name, wc.code, wc.region, gdp,
RANK () OVER (ORDER BY gdp DESC) AS gdp_rank FROM gdp_clean
INNER JOIN world_clean AS wc ON gdp_clean.Code = wc.Code
ORDER BY gdp DESC
LIMIT 20
""")

display(gdp_top20)



emission_gdp_top20 = sql("""
SELECT gdp_top20.name, gdp_top20.code, gdp_top20.region, emissions_per_gdp,
RANK () OVER (ORDER BY emissions_per_gdp DESC) AS emissions_per_gdp_rank FROM gdp_top20
INNER JOIN emissions_gdp ON emissions_gdp.Code = gdp_top20.code
ORDER BY emissions_per_gdp
LIMIT 20
""")

display(emission_gdp_top20)

In [ ]:
fig = px.bar(gdp_top20, x='emissions_per_gdp', y='code', color='region', orientation='h', title='The Enviromental Efficiency of the Economics of top 20 Countries for GDP')
fig.update_layout(yaxis={'categoryorder': 'array', 'categoryarray': emission_gdp_top20['code'].tolist()[::-1]}, width=800, height=800) # Reverse the array to put lowest emissions_per_gdp at the top
fig.show()

In [ ]:

share_top20 = sql("""
SELECT gdp_top20.name, gdp_top20.code, gdp_top20.region,share_clean.renewable,
RANK () OVER (ORDER BY emissions_per_gdp DESC) AS emissions_per_gdp_rank FROM gdp_top20
INNER JOIN emissions_gdp ON emissions_gdp.Code = gdp_top20.code
LEFT JOIN share_clean ON share_clean.Code = gdp_top20.code
ORDER BY share_clean.renewable DESC
LIMIT 20
""")

display(share_top20)

## Top 10 countries for renewable energy

In [ ]:
fig1 = px.bar(share_clean,
             x='renewable',
             y='name',
             color='region',
             orientation='h',
             title='Top 10 countries for renewable share',
             text='renewable')

fig1.update_traces(texttemplate='%{text:.2s}%', textposition='inside') # Position text inside the bar
fig1.update_layout(yaxis={'categoryorder':'total ascending', 'title': ''}, # Remove y-axis title
                    xaxis={'title': ''},
                    width=1000, height=1000, font=dict(size=15),
                    title_x=0.5) # Center the title
fig1.show()

## Top 10 countries for fossil energy

In [ ]:
fig2 = px.bar(fossil_share_clean,
             x='renewable',
             y='name',
             color='region',
             orientation='h',
             title='Top 10 countries for renewable share',
             text='renewable')

fig2.update_traces(texttemplate='%{text:.2f}%', textposition='inside') # Position text inside the bar
fig2.update_layout(yaxis={'categoryorder':'total ascending', 'title': ''}, # Remove y-axis title
                    xaxis={'title': ''},
                    width=1000, height=1000, font=dict(size=15),
                    title_x=0.5) # Center the title
fig2.show()

## Emission and cons 2d map

In [ ]:
fig = px.scatter(merged_data, x='emissions_per_gdp', y='consumption_per_gdp', color='region', hover_data=['Code'])
fig.update_traces(marker=dict(size=12)) # Increase point size
fig.update_layout(width=800, height=800) # Make the plot square
fig.show()

## Choreplot emission

In [ ]:
fig = px.choropleth(emission_clean, locations='Code', color='emissions_total',
                    hover_name='Code', # show country name on hover
                    color_continuous_scale=px.colors.sequential.Greens, # color scale
                    title='CO2 Emissions per Country (2024)')
fig.show()

In [ ]:
europe_emissions = emission_clean[emission_clean['region'] == 'Europe']
display(europe_emissions)

In [ ]:
fig_europe = px.choropleth(europe_emissions, locations='Code', color='emissions_total',
                    hover_name='Code', # show country name on hover
                    color_continuous_scale=px.colors.sequential.Greens, # color scale
                    title='CO2 Emissions per European Country (2024)',
                    fitbounds='locations') # Automatically zoom to fit the European countries
fig_europe.show()

## Top Countries for Renewable Share (by Source) v1 e v2

In [ ]:
renew_cols = ['hydro', 'nuclear', 'wind', 'solar', 'other_renewables', 'biofuels']

share_clean_melted = pd.melt(share_clean,
                             id_vars=['name', 'region', 'Code'],
                             value_vars=renew_cols,
                             var_name='renewable_source',
                             value_name='share_value')

share_clean_melted = share_clean_melted[share_clean_melted['share_value'] != 0]

display(share_clean_melted.head())

In [ ]:
color_map = {
    'hydro': '#1F77B4',  # Vibrant Blue
    'nuclear': '#9467BD', # Vibrant Purple
    'wind': '#17BECF',    # Vibrant Cyan
    'solar': '#FFD700',   # Vibrant Gold/Yellow
    'other_renewables': '#2CA02C', # Vibrant Green
    'biofuels': '#8C564B' # Vibrant Brown
}

fig3 = px.bar(share_clean_melted,
             x='share_value',
             y='name',
             color='renewable_source',
             orientation='h',
             title='Top Countries for Renewable Share (by Source)',
             category_orders={'renewable_source': ['hydro', 'nuclear', 'wind', 'solar', 'other_renewables', 'biofuels']},
             color_discrete_map=color_map) # Apply custom color map

fig3.update_traces(textposition='none') # Remove text from inside the bar
fig3.update_layout(yaxis={'categoryorder':'total ascending', 'title': ''},
                    xaxis={'title': ''},
                    width=1000, height=1000, font=dict(size=15),
                    title_x=0.5,
                    legend_title_text='Renewable Sources') # Add title for the primary legend
fig3.show()

In [ ]:
region_pastel_color_map = {
    'Europe': '#CBD5E8',  # Light Blue
    'Oceania': '#B3E2CD',  # Light Green
    'Americas': '#FDCDAC', # Light Orange
    'Asia': '#F4CAE4',    # Light Pink
    'Africa': '#E6F5C9',  # Pale Yellow Green
    None: '#CCCCCC' # Grey for any missing region
}

shapes_to_add = []

# Get the list of countries in the order they appear on the y-axis
# y_categories_ordered and y_category_to_pos are already available in the kernel state

for i, country_name in enumerate(y_categories_ordered):
    # Find the region for the current country
    country_region = share_clean[share_clean['name'] == country_name]['region'].iloc[0]

    # Get the corresponding pastel color
    fill_color = region_pastel_color_map.get(country_region, '#CCCCCC') # Default to grey if region not found

    # Create a rectangle shape for the 'shadow'
    shapes_to_add.append(
        go.layout.Shape(
            type="rect",
            xref="x",
            yref="y",
            x0=0, # Start at the beginning of the x-axis
            y0=i - 0.5, # Slightly below the bar center
            y1=i + 0.5, # Slightly above the bar center
            x1=100, # End at the maximum of the x-axis (total percentage)
            fillcolor=fill_color,
            layer="below", # Place behind the bars
            line_width=0, # No border for the shadow
            opacity=0.6 # Adjust opacity if desired
        )
    )

# Update the layout of fig3 with the new shapes
fig3.update_layout(shapes=shapes_to_add)

# Display the modified figure
fig3.show()

## Top 20 countries by emission per gdp

In [ ]:
fig = px.bar(merged_data,
             x='emissions_per_gdp',
             y='Code',
             color='region',
             orientation='h',
             title='Top 20 Countries by Emissions per GDP',
             text='emissions_per_gdp')

fig.update_traces(texttemplate='%{text:.2e}', textposition='outside')
fig.update_layout(yaxis={'categoryorder':'total ascending', 'title': ''},
                    xaxis={'title': 'Emissions per GDP'},
                    width=1000, height=800, font=dict(size=15),
                    title_x=0.5)
fig.show()

## TREEMAP

In [ ]:
color_map = {
    'Asia': 'yellow',
    'Africa': 'brown',
    'Oceania': 'lightblue',
    'Americas': 'red',
    'Europe': 'blue',
    'None': 'grey' # Default color for any region not specified
}

# Calculate the percentage of emissions_per_gdp for each entry
total_emissions_per_gdp = emissions_gdp['emissions_per_gdp'].sum()
emissions_gdp['emissions_per_gdp_percentage'] = (emissions_gdp['emissions_per_gdp'] / total_emissions_per_gdp) * 100

fig = px.treemap(emissions_gdp,
                 path=[px.Constant("World"), 'region', 'name'], # Use 'name' for country display
                 values='emissions_per_gdp',
                 title='CO2 Emissions per GDP by Region and Country',
                 color='region', # Color by region
                 color_discrete_map=color_map, # Apply the custom color map
                 hover_data={'emissions_per_gdp_percentage': ':.2f'} # Display percentage on hover with 2 decimal places
                 )

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25), width=800, height=800)

# Customizing hovertemplate to show percentage and original value
fig.update_traces(hovertemplate='<b>%{label}</b><br>Emissions per GDP: %{value}<br>Percentage: %{customdata[0]}%<extra></extra>')

fig.show()

In [ ]:
color_map = {
    'Asia': px.colors.qualitative.Set2[5],
    'Africa': px.colors.qualitative.Set2[6],
    'Oceania': px.colors.qualitative.Set2[0],
    'Americas': px.colors.qualitative.Set2[1],
    'Europe': px.colors.qualitative.Set2[2],
}

# Calculate percentage relative to the region total emissions for 'emission_clean'
emission_clean['region_total_emissions'] = emission_clean.groupby('region')['emissions_total'].transform('sum')
emission_clean['emissions_percentage'] = (emission_clean['emissions_total'] / emission_clean['region_total_emissions']) * 100

fig = px.treemap(emission_clean,
                 path=['region', 'Code'],
                 values='emissions_total',
                 title='CO2 Emissions by Region and Country (Top 20)',
                 color='region', # Specify 'region' as the column to color
                 color_discrete_map=color_map,
                 hover_data=['emissions_percentage'] # Make percentage available for hover
                )

fig.update_layout(margin = dict(t=50, l=25, r=25, b=25), width=800, height=800)

# Customizing hovertemplate to show label, emissions total, and percentage
fig.update_traces(hovertemplate='<b>%{label}</b><br>Emissions: %{value}<br>Percentage: %{customdata[0]:.2f}%<extra></extra>')

fig.show()

## EMISSION GDP

In [ ]:
global_emissions_gdp = sql("""
SELECT
    wc.name,
    wc.code,
    wc.region,
    er.emissions_total,
    gr.ny_gdp_mktp_pp_kd AS gdp,
    (CAST(er.emissions_total AS REAL) / gr.ny_gdp_mktp_pp_kd) AS emissions_per_gdp
FROM
    world_clean AS wc
INNER JOIN
    emission_raw AS er ON wc.code = er.Code
INNER JOIN
    gdp_raw AS gr ON wc.code = gr.Code
WHERE
    er.Year = 2024 AND gr.Year = 2024

ORDER BY emissions_per_gdp DESC
""")

display(global_emissions_gdp)

In [ ]:
min_emissions_per_gdp = global_emissions_gdp['emissions_per_gdp'].min()
max_emissions_per_gdp = global_emissions_gdp['emissions_per_gdp'].max()

print(f"Global Minimum Emissions per GDP: {min_emissions_per_gdp}")
print(f"Global Maximum Emissions per GDP: {max_emissions_per_gdp}")

In [ ]:
plot_data = global_emissions_gdp.copy()
plot_data['emissions_per_gdp_normalized'] = (plot_data['emissions_per_gdp'] - min_emissions_per_gdp) / (max_emissions_per_gdp - min_emissions_per_gdp)

# Select the top 20 countries based on original emissions_per_gdp
plot_data = plot_data.sort_values(by='emissions_per_gdp', ascending=False).head(20)

display(plot_data.head())

In [ ]:
fig = px.bar(plot_data,
             x='emissions_per_gdp_normalized',
             y='name',
             color='region',
             orientation='h',
             title='Top 20 Countries by Normalized CO2 Emissions per GDP (2024)',
             text='emissions_per_gdp_normalized')

fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(yaxis={'categoryorder':'total ascending', 'title': ''},
                    xaxis={'title': 'Normalized Emissions per GDP'},
                    width=1000, height=800, font=dict(size=15),
                    title_x=0.5)
fig.show()

## Bump plot trial

In [ ]:
gdp_top20['gdp_rank'] = gdp_top20['gdp'].rank(ascending=False).astype(int)
display(gdp_top20.head())

In [ ]:
merged_ranks = pd.merge(gdp_top20[['name', 'region', 'gdp_rank']],
                        emissions_rank_gdp_top20[['name', 'emissions_per_gdp_rank']],
                        on='name',
                        how='inner')
display(merged_ranks.head())

In [ ]:
melted_ranks = merged_ranks.melt(id_vars=['name', 'region'],
                                 value_vars=['gdp_rank', 'emissions_per_gdp_rank'],
                                 var_name='rank_type',
                                 value_name='rank_value')

display(melted_ranks.head())

melted_ranks['rank_value'] = pd.to_numeric(melted_ranks['rank_value'])
display(melted_ranks.info())

# Create a list of text positions based on rank_type
text_positions = ['middle left' if rt == 'gdp_rank' else 'middle right' for rt in melted_ranks['rank_type']]

fig = px.line(melted_ranks,
              x='rank_type',
              y='rank_value',
              color='region',
              line_group='name',
              text='name',  # Set text to country name here
              hover_name='name',
              title='Shifts in Country Rankings: GDP vs. Emissions per GDP',
              labels={'rank_type': 'Rank Type', 'rank_value': 'Rank'})

fig.update_layout(yaxis={'autorange': 'reversed'}, width=800, height=800) # Invert y-axis and make it square
fig.update_traces(mode='lines+markers+text', marker=dict(size=12), # Increase marker size
                  line=dict(width=7), # Make lines thicker
                  textposition=text_positions) # Use dynamic text positions
fig.show()

# Other db

In [ ]:
pop_raw = pd.read_csv("https://ourworldindata.org/grapher/population-with-un-projections.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
#pop_metadata = requests.get("https://ourworldindata.org/grapher/population-with-un-projections.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

pop_clean = sql("""
SELECT pop_raw.Code, population__sex_all__age_all__variant_medium as population FROM pop_raw
INNER JOIN world_clean ON pop_raw.Code = world_clean.code
WHERE year = 2024
""")

display(pop_clean)

In [ ]:
path = kagglehub.dataset_download("pralabhpoudel/world-energy-consumption")
cons_raw = pd.read_csv(f"{path}/World Energy Consumption.csv")
#display(cons_raw)




cons_clean = sql("""
SELECT * FROM cons_raw
INNER JOIN world_clean ON cons_raw.iso_code = world_clean.code
WHERE year
""")
display(cons_clean)

In [ ]:
share_raw = pd.read_csv("https://ourworldindata.org/grapher/share-of-primary-energy-consumption-by-source.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
#metadata = requests.get("https://ourworldindata.org/grapher/share-of-primary-energy-consumption-by-source.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

fossil_columns = ['gas', 'oil', 'coal']
renew_cols = ['wind', 'hydro', 'other_renewables', 'nuclear', 'solar', 'biofuels']

new_columns = {}
for col in share_raw.columns:
    if '__pct_direct_primary_energy' in col:
        new_name = col.replace('__pct_direct_primary_energy', '')
        new_columns[col] = new_name
share_raw = share_raw.rename(columns=new_columns)

share_clean= sql(f"""
SELECT  wc.name, wc.region, share_raw.Code, {",".join(fossil_columns)}, {",".join(renew_cols)}, {"+".join(fossil_columns)} as fossil, {"+".join(renew_cols)} as renewable FROM share_raw
INNER JOIN pop_clean on share_raw.Code = pop_clean.Code
INNER JOIN world_clean AS wc ON share_raw.Code = wc.Code
WHERE share_raw.Code IS NOT NULL AND share_raw.Code IS NOT "OWID_WRL"
AND year = 2024

ORDER BY renewable DESC
LIMIT 20
""")

display(share_clean)

fossil_share_clean= sql(f"""
SELECT  wc.name, wc.region, share_raw.Code, {",".join(fossil_columns)}, {",".join(renew_cols)}, {"+".join(fossil_columns)} as fossil, {"+".join(renew_cols)} as renewable FROM share_raw
INNER JOIN pop_clean on share_raw.Code = pop_clean.Code
INNER JOIN world_clean AS wc ON share_raw.Code = wc.Code
WHERE share_raw.Code IS NOT NULL AND share_raw.Code IS NOT "OWID_WRL"
AND year = 2024

ORDER BY fossil DESC
LIMIT 20
""")

display(fossil_share_clean)

emissions_gdp = sql("""
SELECT ec.Code, wc.name, wc.region,
(CAST(ec.emissions_total AS REAL) / gc.gdp) AS emissions_per_gdp,
(CAST(ecc.total_consumption AS REAL) / gc.gdp) AS consumption_per_gdp FROM emission_clean AS ec
INNER JOIN energy_cons_clean AS ecc ON ec.Code = ecc.Code
INNER JOIN gdp_clean AS gc ON ec.Code = gc.Code
INNER JOIN world_clean AS wc ON ec.Code = wc.Code
ORDER BY emissions_per_gdp DESC
""")

display(emissions_gdp)

In [ ]:
energy_cons_raw = pd.read_csv("https://ourworldindata.org/grapher/energy-consumption-by-source-and-country.csv?v=1&csvType=full&useColumnShortNames=true", storage_options = {'User-Agent': 'Our World In Data data fetch/1.0'})
#metadata = requests.get("https://ourworldindata.org/grapher/energy-consumption-by-source-and-country.metadata.json?v=1&csvType=full&useColumnShortNames=true").json()

columns_to_sum = [col for col in energy_cons_raw.columns if col not in ['Year', 'Code', 'Entity']]
energy_cons_raw['total_consumption'] = energy_cons_raw[columns_to_sum].sum(axis=1)
#display(energy_cons_raw)

energy_cons_clean = sql("""
SELECT energy_cons_raw.code, total_consumption FROM energy_cons_raw
INNER JOIN gdp_clean ON energy_cons_raw.Code = gdp_clean.code
WHERE year = 2024
""")

display(energy_cons_clean)